- enviroment


In [1]:
import gensim
import spacy
from scipy import spatial
import json
import numpy as np
import nltk
import math
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from gensim import models
import string
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from collections import defaultdict
from math import log
from collections import Counter
import urllib
from sklearn import metrics

- data file

In [2]:
train_set = "/Users/alfredchen/Documents/GitHub/query-system/data/training.json"
dev_set = "/Users/alfredchen/Documents/GitHub/query-system/data/devel.json"
test_set = "/Users/alfredchen/Documents/GitHub/query-system/data/testing.json"
doc_file = "/Users/alfredchen/Documents/GitHub/query-system/data/documents.json"

#save word embeddings
embeddings = "/Users/alfredchen/Documents/GitHub/query-system/models/mymodel-size"

# save qa log
log_file = "/Users/alfredchen/Documents/GitHub/query-system/data/log.txt"
answer_type_file = "/Users/alfredchen/Documents/GitHub/query-system/data/answer_type_label.txt"
topk_file = "/Users/alfredchen/Documents/GitHub/query-system/data/topk.json"
answer_file = "/Users/alfredchen/Documents/GitHub/query-system/data/answer.txt"
ner_jar = "/Users/alfredchen/Documents/GitHub/query-system/data/stanford-ner.jar"
ner_model = "/Users/alfredchen/Documents/GitHub/query-system/data/english.muc.7class.distsim.crf.ser.gz"

# Part I: preprocess
    - sentence segmentation
    - tokenization
    - word level normalization
    - creat paragraph_index
    - create index2paragraph

In [90]:
    def get_stopwords(flag):
        """method for constructing stopword lookup dictionary
        param: flag: control whether to remove "what.." token from stopwords
        return: a dictionary of stop words
        """
        words = list(nltk.corpus.stopwords.words())
        if flag == False:
        #some stopwords are helpful in targeting answer type, shall not be removed
            words.remove('what')
            words.remove('where')
            words.remove('when')
            words.remove('who')
            words.remove('how')
            words.remove('which')
            # add more #
        stopwords = {}
        for word in words:
            stopwords[word] = stopwords.get(word,0) + 1
        stopwords['one'] = stopwords.get('one',0) + 1
        stopwords['two'] = stopwords.get('one',0) + 1
        stopwords['three'] = stopwords.get('one',0) + 1
        stopwords['four'] = stopwords.get('one',0) + 1
        stopwords['five'] = stopwords.get('one',0) + 1
        stopwords['six'] = stopwords.get('one',0) + 1
        stopwords['seven'] = stopwords.get('one',0) + 1
        stopwords['eight'] = stopwords.get('one',0) + 1
        stopwords['nine'] = stopwords.get('one',0) + 1
        stopwords['ten'] = stopwords.get('one',0) + 1
        return stopwords
    
stopwords = get_stopwords(False)   # do not contain what/where/when etc...
stop_words = get_stopwords(True)   # complete stopwords
nlp = spacy.load('en_core_web_sm') # leverage open source spacy to 

In [4]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    """
    param: word: token
    return: lemmatized token
    """
    lemma = lemmatizer.lemmatize(word, 'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word, 'n')
    return lemma

def removePunc(string):
    """remove punctuation in a string
    param: string: token, sentence
    return: normalized string
    """
    # define punctuation
    punctuations = '''!()[]{};:"\/?@^&*_~'''
    
    # remove punctuation from the string
    no_punct = ""
    for char in string:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

def preprocess_docs(corpus):
    """normalize corpus and build index to paragraph lookup
    param: corpus: wikipedia documents
    return: new_corpus: normalized corpus
    return: para2index: using normalized paragraph to get it's index
    return: index2para: using index to get normalized paragraph
    return rpara2index: using original paragraph to get it's index
    return: index2rpara: using index to get original paragraph
    """
        para2index = {}
        index2para = {}
        index2rpara = {}
        rpara2index = {}
        new_corpus = []
        para_len = 0
        para_count = 0
        for _id, doc in enumerate(corpus):
            new_doc = []
            for _para,para in enumerate(doc['text']):
                rpara2index[para] = (_id,_para)         # para index
                index2rpara[(_id,_para)] = para         # doc_id, answer_para index
                new_para = []
                tem_para_list = []
                docu = nlp(para)
                sents = [x.text for x in docu.sents]
                #sents = para.split('.')
                para_count += 1
                for _sent,sent in enumerate(sents):
                    new_sent=[]
                    words = sent.split(' ')
                    for word in words:
                        new_word = lemmatize(removePunc(word.lower()))
                        if stop_words.get(new_word) or new_word == '':
                            continue
                        para_len += 1
                        new_sent.append(new_word)
                        tem_sent = ' '.join(new_sent)
                    new_para.append(new_sent)
                    tem_para_list.append(tem_sent)
                tem_para = ' '.join(tem_para_list)
                para2index[tem_para] = (_id,_para)         # para index
                index2para[(_id,_para)] = tem_para         # doc_id, answer_para index
                new_doc.append(new_para)
            new_corpus.append(new_doc)
        average_para_len = (para_len/para_count)
        print("average length of normalized paragraph:")
        print(average_para_len)
        return new_corpus, para2index, index2para, rpara2index, index2rpara

In [5]:
docs = json.load(open(doc_file))
corpus,para2index,index2para,rpara2index,index2rpara = preprocess_docs(docs)

69.14823841059602


In [7]:
def train_w2v(corpora,size,iter):
    """method for training word embeddings. 
    No need to called this method, but load a pre-trained word2vec instead
    param: corpora: normalized wikipedia corpus
    param: size: word embedding feature size
    param: iter: training iteration
    return: word2vec model
    """
    docs = corpora
    sentences = []
    for doc in docs:
        for sents in doc:
            new_para = []
            for sent in sents:
                new_para += sent
            sentences.append(new_para)
    model = Word2Vec(sentences, size=size, window=5,iter=iter,workers=4)
    model.save(embeddings+str(size)+'-iter'+str(iter))
    return model

In [158]:
#model = train_w2v(corpus,160,150)   # train the model
#load pre-trained model
model = gensim.models.Word2Vec.load(embeddings+str(160)+'-iter'+str(150))
model = model.wv

In [9]:
# test our pre-trained word2vec model
print(model.most_similar_cosmul(positive=['male', 'queen'], negative=['female']))
print("")
print(model.most_similar(['movie']))


[('king', 0.7751615643501282), ('balmoral', 0.748808741569519), ('duke', 0.7470282912254333), ('fief', 0.7448990941047668), ('royal', 0.7361407279968262), ('lambert', 0.7349075078964233), ('controversially', 0.7227569818496704), ('prince', 0.7206754088401794), ('frederick', 0.7180166244506836), ('elizabeth', 0.7173427939414978)]

[('film', 0.6140520572662354), ('spielberg', 0.43539032340049744), ('console', 0.4009501338005066), ('sylvester', 0.3827870786190033), ('animate', 0.3810267746448517), ('tv', 0.37869757413864136), ('entertainment', 0.37179452180862427), ('stallone', 0.36797311902046204), ('tudio', 0.3666054904460907), ('television', 0.3636029362678528)]


# Part II: Information Retrieval
    - create bm25 to retrieval top N relevant paragraphs to query
    - rank paragraphs
    
We created two method in relevant paragraph retrieval. One is based on distributed semantics using word2vec, another one is BM25. We tested top10 accuracy of two methods in top 1000 paragraphs in development set. Finally, we decide to take BM25.

In [10]:
#get TF 
def term_freqs(docid):
    """getting term frequency on individual documents
    param: docid: document id
    """
    paragraphs = [index2para.get((_id,_para)) for (_id,_para) in index2para.keys() if _id == docid]
    tfs = defaultdict(dict)
    tfs_forward = defaultdict(dict)
    doc_id = 0
    for paragraph in paragraphs:
        #temp = nlp(paragraph)
        tokens = paragraph.split(' ')
        tokens = [x.strip('.').strip(',').strip('?') for x in tokens]
        for token in tokens:
            token = lemmatize(token.lower())
            if not stop_words.get(token):
                term = token
                tfs[term][doc_id] = tfs[term].get(doc_id, 0) + 1 
                tfs_forward[doc_id][term] = tfs[doc_id].get(term, 0) + 1 
        doc_id += 1
    return tfs,doc_id+1,tfs_forward

In [11]:
#build TF_IDF model
def get_tfidf(tfs, total_docment,tfs_forward):
    
    para_length = {}
    for _id,para_list in tfs_forward.items():
        length = 0
        for term, freq in para_list.items():
            length += freq ** 2
        length = length **0.5
        para_length[_id] =  length
    tfidf = defaultdict(dict)
    for term, para_list in tfs.items():
        df = len(para_list)
        for _id, freq in para_list.items(): 
            tfidf[term][_id] = (float(tfs[term][_id]) * log(total_docment / df))# / document_length[doc_id]
    return tfidf

In [12]:
def get_okapibm25(tf, total_docment, docid):
    '''Calculate and return term weights based on okapibm25'''
    k1, b, k3 = 1.5, 0.5, 0
    okapibm25 = defaultdict(dict)
    
    documents = [index2para.get((_id,_para)) for (_id,_para) in index2para.keys() if _id == docid]
    

    # calculate average doc length 
    total = 0
    for d in documents:
        total += len(d)
    avg_doc_length = total/len(documents)*1.0

    for term, doc_list in tf.items():
        df = len(doc_list)
        for doc_id, freq in doc_list.items():
            # term occurences in query
            # qtf = question.count(term) # SEPCIAL 
            qtf = 1.2
            idf = log((total_docment-df+0.5) / df+0.5)
            tf_Dt = ((k1+1)*tf[term][doc_id]) / (k1*((1-b)+b*(len(documents[doc_id])/avg_doc_length) + tf[term][doc_id]))
            if qtf == 0:
                third = 0
            else:
                third = ((k3+1)*qtf) / (k3+qtf)
                okapibm25[term][doc_id] = idf*tf_Dt*third

    return okapibm25

In [13]:
def tfidf_by_doc():
    """construct the bm25 term weight lookup
    """
    index2tfidf = {}
    for i in range(440):
        tfs,total_document,tfs_forward = term_freqs(i)
        #index2tfidf[i] = get_tfidf(tfs, total_docment,tfs_forward)
        index2tfidf[i] = get_okapibm25(tfs, total_document, i)
    return index2tfidf
        
# construct the bm25 term weight lookup
index2tfidf = tfidf_by_doc()

In [14]:
def getTopK(k,query,docid):
    """Retrieve topk paragraphs
    param: k: how many paragraph to be retrieved
    param: query: query
    param: docid: document id
    return: topk ranked relevant paragraph index
    """
    top_document_id = Counter()
    tfidf = index2tfidf.get(docid)
    doc = nlp(query)
    tokens = [lemmatize(x.text.lower()) for x in doc]
    for token in tokens:
        if not stop_words.get(token):
            try:
                term_tfidf = tfidf.get(token)
                for _para, weight in term_tfidf.items():
                     top_document_id[(docid,_para)] += weight
            except:
                continue
            #for _para, weight in term_tfidf.items():
                 #top_document_id[(docid,_para)] += weight
    top_document=[]
    top_document_id = top_document_id.most_common(k)
    for content,weight in top_document_id:
        top_document.append(content)
    return top_document

In [15]:
print("top1 relevant paragraph:")
doc = getTopK(1,'Modern browser support standards-based and defacto what?',410)
print(doc)

[(410, 9)]


In [154]:
#word2vec word index
index2word_set = set(model.index2word)
    
def get_doc_tfidf(docid):
        tfidf = index2tfidf.get(docid)
        return tfidf
    
def get_weight(word,docid,para_id):
        """get term weight in a paragraph using bm25
        """
        tfidf = get_doc_tfidf(docid)
        try:
            weight = tfidf[word][para_id]
        except:
            weight = 0
        return weight
    
def sent2vec(sentence, model, size, index2word_set, docid, paraid):
        """transform word embeddings to sentence embedding
        param: sentence: sentence that want to be transformed
        param: model: pre-trained word embeddings
        param: size: feature vector dimension
        param: index2word_set
        return: transformed sentence vector
        """        
        try:
            doc = nlp(sentence)
            words = [x.text for x in doc]
            #print(words)
        except:
            words = sentence
            #pass
       
        feature_vec = np.zeros((size,), dtype='float32')
        n_words = 0
        #tfidf = get_doc_tfidf(docid)
        for word in words:
            word = word.lower()
            word = lemmatize(word)
            if stop_words.get(word):
                continue
            if word in index2word_set:
                n_words += 1
                if docid != 'disable':
                    w = get_weight(word,docid,paraid)
                else:
                    w = 1
                try:
                    feature_vec = np.add(feature_vec, model[word]*w)# weighted similarity using bm25
                except:
                    pass
        if (n_words > 0):
            feature_vec = np.divide(feature_vec, n_words)
        return feature_vec

In [17]:
def sentence_similarity(v1: np.ndarray, v2: np.ndarray) -> float:
    """compute similarity of two string using cosine sim and embedding
    param: v1: first string
    param: v2: second string
    return: similarity score
    """
    num = np.dot(v1, v2)
    d1 = np.dot(v1, v1)
    d2 = np.dot(v2, v2)
    if d1 > 0.0 and d2 > 0.0:
        return num / math.sqrt(d1 * d2)
    else:
        return 0

In [112]:
def getTopN(N,query,docid):
    """method for retrieve query relevant paragraphs using distributed semantics.
    param: N: number of paragraphs to be retrieved
    param: query: query
    param: docid: document id
    return: top N ranked relevant paragraphs
    """
    q = sent2vec(query, model, 160, index2word_set,'disable','disable')
    paras = [para for (doc,para) in index2para.keys() if doc == docid]
    sim2index = {}
    
    for _para in paras:
        answer = index2para.get((docid,_para))
        #q = sent2vec(query, model, 50, index2word_set,docid,_para)
        answer = sent2vec(answer, model, 160, index2word_set,'disable','disable')
        sim = sentence_similarity(q, answer)
        sim2index[sim] = (docid,_para)
    
    similarity_ranks = sorted(sim2index.keys(),reverse=True)   #most similar
    rank = []
    for i in range(N):
        try:
            rank.append(sim2index.get(similarity_ranks[i]))
        except IndexError as e:
            print(similarity_ranks)
    return rank

index = getTopN(10,
                "A kilogram could be definined as having a Planck constant of what value?",0)    
print(index)   

[(0, 5), (0, 1), (0, 16), (0, 23), (0, 13), (0, 11), (0, 17), (0, 2), (0, 10), (0, 14)]


Test Top10 retrieval accuracy of BM25 and distributed semantic method

In [120]:
def gold_standard(dataset):
    file = json.load(open(dataset))
    gold2index={}
    for line in file:
        _id = line['docid']
        try:
            _para = line['answer_paragraph']
        except KeyError:
            _para = line['id']
        gold2index[line['question']]=(_id,_para)
    return gold2index

def my_qa(n,dataset,flag):
    file = json.load(open(dataset))
    qa2index={}
    for i in range(n):
        save = open(log_file,"a")
        line = file[i]
        query = line['question']
        if flag == 1:
            possible = getTopK(1,query,line['docid'])
        elif flag == 2:
            possible = getTopN(1,query, line['docid'])
        qa2index[line['question']] = possible
        record = line['question']+':'+str(possible)+'\n'
        save.write(record)
        save.close()
    return qa2index

In [121]:
def acc(gold_standard,qas):
    query = qas.keys()
    total = 0
    correct = 0
    for i,q in enumerate(query):
        gold = gold_standard.get(q)
        if gold in qas.get(q):
            correct += 1
        total += 1
    acc = correct/total
    return acc

In [122]:
#test on top 100 devset
print("accurcacy of semantic:")
gold_index = gold_standard(dev_set)
qa_index = my_qa(100,dev_set,2)
accu = acc(gold_index,qa_index)
print(accu)

accurcacy of semantic:
0.56


In [123]:
#test on top 100 devset
print("accurcacy of BM25:")
gold_index = gold_standard(dev_set)
qa_index = my_qa(100,dev_set,1)
accu = acc(gold_index,qa_index)
print(accu)

accurcacy of BM25:
0.83


# Part III: Named Entity Recognition and Answer Type Classification

This part, we focused on extract potential answer terms from retrieved relevant paragraphs. We leverage open source spacy in helping us doing named entity recognition, noun-phrase chunk extraction. We use machine learning to classify answer type of query using top 18174 training data. The feature of this task is distributed semantics (word embedding). The answer types are shown below:


    - PERSON              -People, including fictional.
    - NORP	              -Nationalities or religious or political groups.
    - FACILITY	          -Buildings, airports, highways, bridges, etc.
    - ORG	              -Companies, agencies, institutions, etc.
    - GPE	              -Countries, cities, states.
    - LOC	              -Non-GPE locations, mountain ranges, bodies of water.
    - PRODUCT	          -Objects, vehicles, foods, etc. (Not services.)
    - EVENT	              -Named hurricanes, battles, wars, sports events, etc.
    - WORK_OF_ART	      -Titles of books, songs, etc.
    - LAW	Named         -documents made into laws.
    - LANGUAGE	          -Any named language.
    - DATE	              -Absolute or relative dates or periods.
    - TIME	              -Times smaller than a day.
    - PERCENT	          -Percentage, including "%".
    - MONEY	              -Monetary values, including unit.
    - QUANTITY	          -Measurements, as of weight or distance.
    - ORDINAL	          -"first", "second", etc.
    - CARDINAL	          -Numerals that do not fall under another type.


In [145]:
# set up classifier for detect answer type

nlp = spacy.load('en_core_web_sm')
#preprocess word in query
def preprocess_sentence(sentence):
    """method query processing
    param: sentence: original query
    return: processed query
    """
    doc = nlp(sentence)     # tokenization
    words = [x.text for x in doc]
    words = [lemmatize(removePunc(x.lower())) for x in words]   #normalization
    words = [x for x in words if not stopwords.get(x) and x != '']
    return ' '.join(words)

def named_entity_recognize(sentence,valid_ner):
    """method for extract named entity from relevant paragraphs
    param: sentence: texture paragraphs or sentence
    param: valid_ner: type of named entity to extract
    return: a list of named entities
    """
    entities = []
    ner_tag = []
    sentence = removePunc(sentence)
    doc = nlp(sentence)
    # case for tagging training data for answer type classification task
    if valid_ner == 'disable':
        ner_tag = ''
        for ent in doc.ents:
            ner_tag=ent.label_
        if ner_tag == '':
            ner_tag = 'O'
        return ner_tag
    
    # case not 'O'
    for ent in doc.ents:
        if ent.label_ in valid_ner:
            entities.append(lemmatize(removePunc(ent.text.lower())))
            ner_tag.append(ent.label_)
            
    # case 'O', return noun_phrase chunk(start with adj/propn and end with noun)
    if valid_ner == 'O':
        for chunk in doc.noun_chunks:
            #print(chunk.text)
            proc = nlp(chunk.text)
            flag = 1
            for ent in proc.ents:
                if ent:
                    flag = 0

            if flag == 1:
                entities.append(chunk.text.lower())
    if entities == []:
        return 'unknown'
    return entities

In [53]:
# test on the named entity recognize
entities = named_entity_recognize("John Smith, Mike Johnson, January, 11th 2017",['DATE'])  
print(entities)

context = index2rpara.get((14,23))
#print(context)

entities = named_entity_recognize(context,['QUANTITY','PERSON'])
print(entities)

['john smith', 'mike johnson', 'january, 11th 2017']
['essonne', '814 square kilometers']


Then we use ner to tag answers of training data and assume the result is gold standard. We use these tags as labels to perform a supervised machine learning query answer type classification task. In order to justify the mistakes of the ner tags, we also apply a rule to amend labels. For instance, queries containing when is obviously an answer type of 'DATE'.

In [70]:
# apply ner recognition to get all answer type from training data
"""
def tag_answer(dataset):
    file = json.load(open(dataset))
    for _id,line in enumerate(file):
        answer = line['text']
        label = named_entity_recognize(answer,'disable')
        f = open(answer_type,"a")
        f.write(str(_id)+':'+str(label)+'\n')
        f.close()
    print('done')
    return True

tag_answer(train_set)
""" 
pass

In [92]:
rules = {
            'name':'PERSON',
            'country': 'GPE',
            'capital': 'GPE',
            'newspaper':'ORG',
            'company':'ORG',
            'city': 'GPE',
            'person':'PERSON',
            'location': ['LOC','GPE'],
            'mountain':'LOC',
            'website':'ORG',
            'airline':'ORG',
            'which organization': 'ORG',
            'where': 'GPE',
            'when': 'DATE',           
            'who': 'PERSON',     
            'what scientist':'PERSON',
            'what time':'DATE',
            'year':'DATE',
            'what athlete':'PERSON',
            'which athlete':'PERSON',
            'percentage':'PERCENT',
            'what people': 'PERSON',
            'what date':'DATE',
            'what day':'DATE',
            'what year': 'DATE',
            'what city' : 'GPE',
            'which company': 'ORG' ,
            'which publication':'ORG',
            'what government':['ORG','NORP'],
            'which supporters' : 'PERSON',
            'which footballer': 'PERSON',
            'which actor':'PERSON',
            'Which actress':'PERSON',
            'which American actress':'PERSON',
            'what activists':'PERSON',
            'which team member' : 'PERSON',
            'what football star': 'PERSON',
            'which blogger': 'PERSON',
            'which torchbearer':'PERSON',
            'which wheelchair-bound torchbearer' : 'PERSON',
            'how much of': ['CARDINAL','MONEY','QUANTITY','PERCENT'],
            'by how much': ['CARDINAL','MONEY','QUANTITY','PERCENT'],
            'how many':['CARDINAL','QUANTITY','PERCENT'],
            'value':['CARDINAL','MONEY','PERCENT'],
            'rank':['CARDINAL'],
            'how much': 'O'
            
            
}

money_list = ['cost', 'worth', 'spend', 'money', 'worth', 'invest']

def tag_answer_type(question):
    answer_type = 'O'
    processed_question = []
    processed_question_str = None
    for token in [question]:
        processed_question.append(token.lower())
    processed_question_str = " ".join(x for x in processed_question)
    for k,v in rules.items():
        if k in processed_question_str:
            #print(k)
            if k == 'how much':
                for item in money_list:
                    #print("item", item)
                    if item in processed_question_str:
                        answer_type = 'MONEY'
                    else:
                        continue
            else:
                answer_type = rules.get(k, "O")    
    return answer_type

The following two cells show the process of training answer type classifier. Do not need to run these cells, just load a pre-trained classifier in later cells.

In [161]:
# train a classifier to recognize answer type from query using top 18173 data)
def query2vector(dataset):
    """method for transforming texture query into embeddings
    param: dataset: training.json
    return: X: queries feature matrix
    return: Y: labels
    """
    label =  open(answer_type_file)
    train = json.load(open(dataset))
    X = []
    Y = []
    for i,line in enumerate(label):
        #query = preprocess_sentence(train[i]['question'])
        x = sent2vec(train[i]['question'], model, 160, index2word_set, 'disable','disable')
        y = line.split(':')[1].strip()
        X.append(x)
        Y.append(y)
    label.close()
    return X,Y
    
vectors,labels = query2vector(train_set)
print(len(vectors))
print(len(labels)) 

18174
18174


In [162]:
# supervised logistic regression classifier
def train(X,y):
    """using grid search to find best hyperparameter
    """
    # Create regularization penalty space
    penalty = ['l1', 'l2']
    # Create regularization hyperparameter space
    C = np.logspace(0, 4, 10)
    # Create hyperparameter options
    hyperparameters = dict(C=C, penalty=penalty)
    # Create grid search using 5-fold cross validation
    clf = LogisticRegression()
    grid = GridSearchCV(clf, hyperparameters, cv=5, verbose=0)
    # Fit grid search
    best_model = grid.fit(X, y)
    # View best hyperparameters
    print(grid.best_score_)
    return best_model
            
answer_type_clf = train(vectors,labels)
print("done")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


0.7793000990425883
done


Start running from this cell.

In [163]:
#save clf 
import joblib
#joblib.dump(answer_type_clf, '/Users/alfredchen/Documents/GitHub/query-system/models/relevance.pkl',protocol=2)
#load clf
answer_type_clf= joblib.load('/Users/alfredchen/Documents/GitHub/query-system/models/relevance.pkl')

In [164]:
def pred_answer_type(query,classifier):
    """semantic machine learning and rule-based answer type classifier
    """
    q = sent2vec(query, model, 160, index2word_set, 'disable','disable')
    preds = classifier.predict([q][0:])
    if preds[0] == 'O':
        pred=tag_answer_type(query)
    else:
        pred = preds[0]
    return pred

print(pred_answer_type('who first observed the photoelectric effect?',answer_type_clf))

PERSON


# Part VI: Answer Ranking Pipeline
This part, we create a pipeline to process texture query and a list of potential answers to output a single most probable answer. Three filter is contained in the pipeline. Firstly, we will remove answers overlap with query. Secondly, we will remove answers semantically unrelated to query. Thirdly, we set up a co-occurrence method to rank rest answers. If an answer co-occur with many query terms in a sentence boundary, it is more likely to be the correct answer.

In [124]:
#stop_words = get_stopwords(False)
def first_filter(query,answer):
    """filter out answers overlap with query
    param: query
    param: answer: a list of answers
    return: a list of filterd answers
    """
    query = preprocess_sentence(query).split(' ')
    q = []
    a = []
    template = []
    for w in query:
        if not stop_words.get(w):
            lemmatize(removePunc(w.lower()))
            q.append(w)

    for w in answer:
        try:
            flag = 1
            words = w.split(' ')
            #words = [lemmatize(removePunc(x).lower()) for x in words]
            for word in words:
                if not stop_words.get(word):
                    if word not in template:
                        if word in q:
                            flag = 0
                    else:
                        flag = 0
                else:
                    flag = 0
            if flag == 1:
                a.append(w)
                template.append(word)
        except:
            print("exception in first filter")                
    q = ' '.join(q)
    return q,a

In [141]:

def second_filter(query,answer):
    """
    param: query
    param: answer: a list of answers
    return: a list of filterd answers
    """
    
    query, answer = first_filter(query,answer)
    query_semantic = sent2vec(query, model, 160, index2word_set, 'disable','disable')
    max_sim = 0
    potential_answer = []
    sim_answer = ''
    try:        
        for a in answer:
            ans = sent2vec(a, model, 160, index2word_set, 'disable','disable')
            sim = cosine_similarity(query_semantic,ans)
            if sim >= max_sim:
                potential_answer.append(a)
    except:
        print("exception in second filter")
    return query, potential_answer


def co_occurrence(query,answer,context,docid,paraid):
    """
    answer ranking using co_occurrence
    """
    key_words = query.split(' ')
    #tfidf = get_doc_tfidf(docid)
    doc = nlp(context)
    sentences = [x.text for x in doc.sents]
    co_occurrence_score = {}
    for ans in answer:
        score = 200
        try:
            weight = get_weight(ans.split(' ')[-1],docid,paraid)  # noun-phrase, weight on the last token
        except:
            weight = get_weight(ans,docid,paraid)
        for sent in sentences:
            for word in key_words:
                w2 = get_weight(word,docid,paraid)
                try:
                    if word in sent and ans in sent:
                        score -= weight
                except:
                    continue
        co_occurrence_score[score] = ans
    return co_occurrence_score
        
    
def formalize_answers(answer_list):
    """take highest ranking answer
    """    
    similarity_ranks = sorted(answer_list.keys(),reverse=False)   #sims
    rank = []
    for i in range(1):
        try:
            rank.append(answer_list.get(similarity_ranks[i]))
        except IndexError as e:
            result = 'unknown'
    try:
        result = rank[0]
    except:
        result = 'unknown'
    
    result = lemmatize(removePunc(result.lower()))
    final = ''
    if result[0] is '%':
        final += '% '
    else:
        final += result[0]
    for i in range(1,len(result)):
        if result[i] is '%':
            final += ' %'
        else:
            final += result[i]
    return final   

# Part V: Combine Modules

In [136]:
def extract_answer_from_topk(query,topk):
    """method to combine entire retrieval, ner, pipeline
    param: query
    param: topk: a list of paragraph index
    return: one answer
    """
    context = ""
    # paragraph retrieval
    for index in topk:
        context += index2rpara.get(tuple(index))
    # machine learning and rule-based hyibrid NER
    valid_ner = pred_answer_type(query,answer_type_clf)
    #if valid_ner == 'O':
        #valid_ner = tag_answer_type(query)
    # answer extraction
    potential_answers = named_entity_recognize(context,valid_ner)
    # answer ranking
    query, answer = second_filter(query,potential_answers)
    ans = {}
    score = {}
    context = ""
    for index in topk:
        context = index2para.get(index)
        ans.update(co_occurrence(query, answer, context,index[0],index[1]))
    #answer finalization
    final = formalize_answers(ans)
    return final

topk = getTopK(1,"who first observed the photoelectric effect?",0)
print(topk)
query = "who first observed the photoelectric effect?"

answers = extract_answer_from_topk(query,topk)
print(answers)

[(0, 7)]
heinrich hertz


In [168]:
# test acc on devset
import csv

def test_answers(dataset):
    """question answering system
    param: dataset
    return: write into .csv file
    """
    file = json.load(open(dataset))
    gold = []
    pred = []
    csv_file = open('output.csv', 'a')
    writer = csv.writer(csv_file)
    writer.writerow(['id', 'answer'])
    csv_file.close()

    for _id,line in enumerate(file):
        query = line['question']
        topk = getTopK(1,query,line['docid'])
        #print(topk)
        answer = extract_answer_from_topk(query,topk)
        #gold.append(line['text'])
        pred.append(answer)
        #print(query)
        
        csv_file = open('output.csv', 'a')
        writer = csv.writer(csv_file)
        writer.writerow([_id, answer])
        #print(_id)
        if _id % 50 == 0:
            print(_id)
            #print(metrics.accuracy_score(gold,pred))
        csv_file.close()
    return gold,pred

gold,pred = test_answers(test_set)
        

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600


In [ ]:
def print_metrices(y_test,y_pred): 
    print("Accuracy:") 
    print(metrics.accuracy_score(y_test,y_pred))
    
    print("\nAverage precision:")
    print(metrics.precision_score(y_test,y_pred,average='macro'))

    print("\nAverage recall:")
    print(metrics.recall_score(y_test,y_pred,average='macro'))

    print("\nAverage f1:")
    print(metrics.f1_score(y_test,y_pred,average='macro'))

    print("\nClassification report:")
    print(metrics.classification_report(y_test, y_pred))